## License 

Copyright 2021-2025 Patrick Hall (jphall@gwu.edu)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

*DISCLAIMER*: This notebook is not legal or compliance advice.

# Model Evaluation Notebook

#### Imports and inits

In [1]:
import os              # for directory and file manipulation
import numpy as np     # for basic array manipulation
import pandas as pd    # for dataframe manipulation
import datetime        # for timestamp

# for model eval
from sklearn.metrics import accuracy_score, f1_score, log_loss, mean_squared_error, roc_auc_score

# global constants 
ROUND = 3              # generally, insane precision is not needed 
SEED = 12345           # seed for better reproducibility

# set global random seed for better reproducibility
np.random.seed(SEED)

#### Set basic metadata

In [2]:
y_name = 'high_priced'
scores_dir = 'data/scores'

#### Read in score files 

In [3]:
# init score frame with known test y values
scores_frame = pd.read_csv(scores_dir + os.sep +'key.csv', index_col='Unnamed: 0')

# create random folds in reproducible way
np.random.seed(SEED)
scores_frame['fold'] = np.random.choice(5, scores_frame.shape[0])

# read in each score file in the directory as a new column 
for file in sorted(os.listdir(scores_dir)):
    if file != 'key.csv' and file.endswith('.csv'):
        scores_frame[file[:-4]] = pd.read_csv(scores_dir + os.sep + file)['phat']

# sanity check 
scores_frame

,high_priced,fold,Group11_nmodels50_rem_ebm,Group11_s23456_rem_ebm,Group11_s309_rem_ebm,Group11_s78600_rem_ebm,group10_rem_ebm,group2_rem_ebm_01_20_15,group2_rem_ebm_22_17_12,group3_rem_ebm_baseline,group3_rem_ebm_higherAUC,group5_rem_ebm,group6_rem_ebm,group7_rem_ebm,group8_rem_ebm,group9_rem_ebm,ph_advval_rem_ebm_300
0,0.0,2,0.137540,0.139748,0.205271,0.179329,0.171282,0.185777,0.172227,0.143110,0.223292,0.190114,0.135934,0.211464,0.231874,0.217985,0.046616
1,0.0,1,0.087820,0.082908,0.061675,0.050835,0.052447,0.054526,0.048065,0.086134,0.056014,0.050783,0.089782,0.056915,0.050760,0.051602,0.107210
2,1.0,4,0.205859,0.175563,0.138244,0.134298,0.142725,0.115459,0.143908,0.178489,0.144938,0.126908,0.146695,0.145323,0.144645,0.150886,0.205463
3,0.0,1,0.004215,0.005734,0.004596,0.005623,0.012051,0.017398,0.003693,0.005306,0.003517,0.004722,0.009907,0.009820,0.003467,0.003105,0.026128
4,1.0,2,0.157931,0.128220,0.171561,0.131882,0.128083,0.127062,0.134211,0.130522,0.157840,0.145517,0.113684,0.151989,0.159707,0.157856,0.205463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19826,0.0,3,0.160417,0.175563,0.180265,0.222589,0.232189,0.178684,0.235976,0.178489,0.205958,0.185210,0.152040,0.187260,0.202280,0.190782,0.161056
19827,0.0,1,0.091154,0.113400,0.158804,0.130824,0.130660,0.119742,0.131474,0.117766,0.171042,0.139019,0.109661,0.156364,0.167931,0.151061,0.166683
19828,1.0,3,0.118401,0.097393,0.239517,0.202036,0.212546,0.210369,0.215991,0.100339,0.256641,0.216455,0.089782,0.238478,0.234941,0.236455,0.205463
19829,0.0,1,0.000951,0.027918,0.001340,0.023791,0.011642,0.006776,0.000545,0.028972,0.000517,0.000769,0.003152,0.006474,0.000477,0.000084,0.006582


#### Utility function for max. accuracy

In [4]:
def max_acc(y, phat, res=0.01): 

    """ Utility function for finding max. accuracy at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. accuracy, default 0.01.
        :return: Max. accuracy for model scores.
    
    """
    
    # init frame to store acc at different cutoffs
    acc_frame = pd.DataFrame(columns=['cut', 'acc'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find accuracy at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        acc = accuracy_score(temp_df.iloc[:, 0], temp_df['decision'])
        acc_frame = acc_frame.append({'cut': cut,
                                      'acc': acc},
                                     ignore_index=True)

    # find max accurcay across all cutoffs
    max_acc = acc_frame['acc'].max()
    
    # house keeping
    del acc_frame, temp_df
    
    return max_acc

####  Utility function for max. F1

In [5]:
def max_f1(y, phat, res=0.01): 
    
    """ Utility function for finding max. F1 at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. F1, default 0.01.
        :return: Max. F1 for model scores.
    
    """
    
    # init frame to store f1 at different cutoffs
    f1_frame = pd.DataFrame(columns=['cut', 'f1'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find f1 at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        f1 = f1_score(temp_df.iloc[:, 0], temp_df['decision'])
        f1_frame = f1_frame.append({'cut': cut,
                                    'f1': f1},
                                    ignore_index=True)
        
    # find max f1 across all cutoffs
    max_f1 = f1_frame['f1'].max()
    
     # house keeping
    del f1_frame, temp_df
    
    return max_f1

#### Rank all submitted scores 

In [6]:
eval_frame = pd.DataFrame() # init frame to hold score ranking
metric_list = ['acc', 'auc', 'f1', 'logloss', 'mse'] # metric to use for evaluation

# create eval frame row-by-row
for fold in sorted(scores_frame['fold'].unique()): # loop through folds 
    for metric_name in metric_list: # loop through metrics
        
        # init row dict to hold each rows values
        row_dict = {'fold': fold,
                    'metric': metric_name}
        
        # cache known y values for fold
        fold_y = scores_frame.loc[scores_frame['fold'] == fold, y_name]
        
        for col_name in scores_frame.columns[2:]:
            
            # cache fold scores
            fold_scores = scores_frame.loc[scores_frame['fold'] == fold, col_name]
            
            # calculate evaluation metric for fold
            # with reasonable precision 
            
            if metric_name == 'acc':
                row_dict[col_name] = np.round(max_acc(fold_y, fold_scores), ROUND)
                
            if metric_name == 'auc':
                row_dict[col_name] = np.round(roc_auc_score(fold_y, fold_scores), ROUND)
                
            if metric_name == 'f1':
                row_dict[col_name] = np.round(max_f1(fold_y, fold_scores), ROUND) 
                
            if metric_name == 'logloss':
                row_dict[col_name] = np.round(log_loss(fold_y, fold_scores), ROUND)
                
            if metric_name == 'mse':
                row_dict[col_name] = np.round(mean_squared_error(fold_y, fold_scores), ROUND)
        
        # append row values to eval_frame
        eval_frame = eval_frame.append(row_dict, ignore_index=True)

# init a temporary frame to hold rank information
rank_names = [name + '_rank' for name in sorted(eval_frame.columns) if name not in ['fold', 'metric']]
rank_frame = pd.DataFrame(columns=rank_names)        

# set columns to necessary order
eval_frame = eval_frame[['fold', 'metric'] + [name for name in sorted(eval_frame.columns) if name not in ['fold', 'metric']]]

# determine score ranks row-by-row
for i in range(0, eval_frame.shape[0]):
        
        # get ranks for row based on metric
        metric_name = eval_frame.loc[i, 'metric']
        if metric_name in ['logloss', 'mse']:
            ranks = eval_frame.iloc[i, 2:].rank().values
        else:
            ranks = eval_frame.iloc[i, 2:].rank(ascending=False).values
        
        # create single-row frame and append to rank_frame
        row_frame = pd.DataFrame(ranks.reshape(1, ranks.shape[0]), columns=rank_names)
        rank_frame = rank_frame.append(row_frame, ignore_index=True)
        
        # house keeping
        del row_frame

# merge ranks onto eval_frame
eval_frame = pd.concat([eval_frame, rank_frame], axis=1)

# house keeping
del rank_frame
        
eval_frame

,fold,metric,Group11_nmodels50_rem_ebm,Group11_s23456_rem_ebm,Group11_s309_rem_ebm,Group11_s78600_rem_ebm,group10_rem_ebm,group2_rem_ebm_01_20_15,group2_rem_ebm_22_17_12,group3_rem_ebm_baseline,...,group2_rem_ebm_01_20_15_rank,group2_rem_ebm_22_17_12_rank,group3_rem_ebm_baseline_rank,group3_rem_ebm_higherAUC_rank,group5_rem_ebm_rank,group6_rem_ebm_rank,group7_rem_ebm_rank,group8_rem_ebm_rank,group9_rem_ebm_rank,ph_advval_rem_ebm_300_rank
0,0.0,acc,0.900,0.900,0.901,0.901,0.901,0.901,0.901,0.900,...,4.0,4.0,11.5,11.5,11.5,11.5,4.0,11.5,11.5,4.0
1,0.0,auc,0.758,0.731,0.794,0.776,0.775,0.754,0.781,0.741,...,12.0,6.0,13.0,1.0,10.0,15.0,5.0,4.0,3.0,9.0
2,0.0,f1,0.326,0.301,0.353,0.332,0.331,0.312,0.335,0.309,...,12.0,7.0,13.0,3.0,11.0,15.0,6.0,4.5,2.0,4.5
3,0.0,logloss,0.287,0.295,0.273,0.279,0.280,0.287,0.277,0.292,...,11.5,6.0,13.0,2.0,10.0,15.0,7.0,4.5,4.5,2.0
4,0.0,mse,0.083,0.085,0.081,0.081,0.082,0.083,0.081,0.084,...,11.0,4.0,13.0,4.0,11.0,15.0,4.0,4.0,4.0,8.5
5,1.0,acc,0.906,0.906,0.906,0.906,0.906,0.906,0.906,0.906,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
6,1.0,auc,0.739,0.705,0.777,0.746,0.749,0.734,0.756,0.710,...,12.0,7.0,13.0,1.0,10.0,15.0,6.0,2.5,4.0,5.0
7,1.0,f1,0.281,0.262,0.326,0.292,0.296,0.279,0.295,0.265,...,12.0,8.5,13.0,3.0,8.5,15.0,4.0,1.5,5.0,6.0
8,1.0,logloss,0.279,0.288,0.270,0.278,0.278,0.281,0.275,0.287,...,12.0,7.0,13.0,3.0,9.0,15.0,6.0,3.0,3.0,1.0
9,1.0,mse,0.080,0.082,0.078,0.080,0.080,0.080,0.079,0.081,...,10.0,5.5,13.0,5.5,10.0,15.0,5.5,2.0,5.5,2.0


#### Save `eval_frame` as CSV

In [7]:
eval_frame.to_csv('model_eval_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.csv'), 
                  index=False)

#### Display simple ranked score list 

In [8]:
eval_frame[[name for name in eval_frame.columns if name.endswith('rank')]].mean().sort_values()

Group11_s309_rem_ebm_rank          3.34
group3_rem_ebm_higherAUC_rank      3.82
group8_rem_ebm_rank                4.42
group9_rem_ebm_rank                4.92
ph_advval_rem_ebm_300_rank         4.96
group7_rem_ebm_rank                6.14
group2_rem_ebm_22_17_12_rank       6.58
Group11_s78600_rem_ebm_rank        7.52
group10_rem_ebm_rank               8.24
group5_rem_ebm_rank                9.46
group2_rem_ebm_01_20_15_rank      10.18
Group11_nmodels50_rem_ebm_rank    11.10
group3_rem_ebm_baseline_rank      12.36
Group11_s23456_rem_ebm_rank       13.04
group6_rem_ebm_rank               13.92
dtype: float64